## Digitised Montage

This notebook try different montage on the EEG data from LEMON dataset to find out which one is the best fit.'
As a temporary solution, we average the two sets of measurement of ficudial points to make a coordiate to create a montage from digitized EEG locations. However, this approach is not yet validated. So thi notebook will find out the validity of this by visualisation montage on the 3D space.

In [ ]:
# imports
import numpy as np
import scipy.io
import mne
from matplotlib import pyplot as plt
import os.path as op

from src.EEGNet.preprocessing.utils import get_channel_positions, get_headpoints

In [ ]:
# open EEG and headshape data
raw = mne.io.read_raw_brainvision('data/raw/sub-032304/RSEEG/sub-010005.vhdr')
raw.set_channel_types({'VEOG': 'eog'})

# digitized points
data_dir = 'data/EEG_Localizer_BIDS_ID/'
positions = get_channel_positions(data_dir)
headpoints = get_headpoints(data_dir)

# subject's headshape
sub_id = 'sub-010005'
head1 = headpoints[sub_id]['first']
head2 = headpoints[sub_id]['second']
head = [(h1+h2)/2 for h1, h2 in zip(head1.values(), head2.values())]  # if we want to average of the fiducials
pos = positions[sub_id]
hpi = np.array(list(headpoints[sub_id]['second'].values()))

# fsaverage data
fs_dir = 'data/fsaverage'
subjects_dir = op.dirname(fs_dir)

# The files live in:
subject = "fsaverage"
trans = "fsaverage"  # MNE has a built-in fsaverage transformation
src = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
bem = op.join(fs_dir, "bem", "fsaverage-5120-5120-5120-bem-sol.fif")

In [ ]:
# Visualisation
pos_dir = 'data/EEG_Localizer_BIDS_ID/'
sub_id = 'sub-010005'
EEG = mne.io.read_raw_brainvision('data/raw/sub-032304/RSEEG/sub-010005.vhdr')
EEG.set_channel_types({'VEOG': 'eog'})
EEG.rename_channels({'Fp1': 'FP1',
                     'Fp2': 'FP2'})

position = get_channel_positions(pos_dir)[sub_id]
headpoint = get_headpoints(pos_dir)[sub_id]
head1, head2 = headpoint['first'], headpoint['second']
head = np.array([(h1+h2)/2 for h1, h2 in zip(head1.values(), head2.values())])
montage = mne.channels.make_dig_montage(ch_pos=position, nasion=head[0], lpa=head[1], rpa=head[2])
# montage = mne.channels.make_dig_montage(ch_pos=position, nasion=head2['NA'], lpa=head2['LPA'], rpa=head2['RPA'])
# montage = mne.channels.make_standard_montage('standard_1005')

EEG.set_montage(montage)
mne.viz.plot_alignment(EEG.info)

In [ ]:
matfile = scipy.io.loadmat('data/EEG_Localizer_BIDS_ID/sub-010005/sub-010005.mat')
x = matfile['HeadPoints'][0][0][0][0]
y = matfile['HeadPoints'][0][0][0][1]
z = matfile['HeadPoints'][0][0][0][2]
coord = np.vstack((x, y, z)).T
labels = matfile['HeadPoints'][0][0][1][0]
first_set = {l[0]: i for l, i in zip(labels[:3], coord[:3])}
second_set = {l[0]: i for l, i in zip(labels[3:], coord[3:])}

Observations:
1. NA: the two measurments of x coordinates are always positive.
2. NA: the two measurments of y have always opposite signs (There is no pattern of one being positive and the other negative).
3. NA: the two measurments of z have always opposite signs (There is no pattern of one being positive and the other negative).

1. LPA: the two measurments of x coordinates are always positive.

In [ ]:
from mne.channels.layout import _get_ch_info, make_eeg_layout, _find_kit_layout, _find_topomap_coords
from mne import Info
exclude = 'bads'
info = EEG.info

lay = make_eeg_layout(info, exclude=exclude)